# Working with H3 in Vgrid DGGS

[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.gishub.vn/lab/index.html?path=notebooks/vgrid/01_h3.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeoshub/vgrid/blob/master)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeoshub/vgrid/HEAD)

Full Vgrid DGGS documentation is available at [vgrid document](https://vgrid.gishub.vn).

To work with Vgrid DGGS directly in GeoPandas and Pandas, please use [vgridpandas](https://pypi.org/project/vgridpandas/). Full Vgridpandas DGGS documentation is available at [vgridpandas document](https://vgridpandas.gishub.vn).

To work with Vgrid DGGS in QGIS, please use [Vgrid Plugin](https://plugins.qgis.org/plugins/vgridtools/).

## Install vgrid
Uncomment the following line to install [vgrid](https://pypi.org/project/vgrid/) if needed.

In [ ]:
# %pip install vgrid --upgrade

## latlon2h3

In [ ]:
from vgrid.conversion.latlon2dggs import latlon2h3

lat = 10.775276
lon = 106.706797
res = 9
h3_id = latlon2h3(lat, lon, 10)
h3_id

### H3 to Shapely Polygons

In [ ]:
import geopandas as gpd
from vgrid.conversion.dggs2geo.h32geo import h32geo
h3_geo = h32geo(h3_id)
h3_geo

### H3 to GeoJSON

In [ ]:
from vgrid.conversion.dggs2geo.h32geo import h32geojson

h3_geojson = h32geojson(h3_id)
h3_geojson

### Vector to H3

In [ ]:
from vgrid.conversion.vector2dggs.vector2h3 import vector2h3

file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/shape/polygon.geojson"
)
vector_to_h3 = vector2h3(file_path, resolution=10, compact=False, 
                predicate = "centroid_within", output_format="gpd")

# Visualize the output
import folium

m = folium.Map(tiles="CartoDB positron")

h3_layer = folium.GeoJson(
vector_to_h3,
style_function=lambda x: {
    "fillColor": "blue",
    "color": "black",
    "fillOpacity": 0.3,
    "weight": 1,
},
tooltip=folium.GeoJsonTooltip(
    fields=["h3", "resolution", "cell_area"],
    aliases=["H3 ID", "Resolution", "Area (m²)"],
    style="""
        background-color: white;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
),
).add_to(m)

m.fit_bounds(h3_layer.get_bounds())

# Display the map
m

### H3 Compact

In [ ]:
from vgrid.conversion.dggscompact.h3compact import h3compact
h3_compacted = h3compact(vector_to_h3,output_format="gpd")
h3_compacted.plot(edgecolor="white")

### H3 Expand

In [ ]:
from vgrid.conversion.dggscompact.h3compact import h3expand

h3_expanded = h3expand(
    h3_compacted, resolution=11, output_format="gpd"
)   
h3_expanded.plot(edgecolor="white")

### H3 Binning

In [ ]:
from vgrid.binning.h3bin import h3bin
file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/csv/dist1_pois.csv"
)
stats="count"
h3_bin = h3bin(file_path, resolution=10, stats=stats, 
                # numeric_field="confidence",
                # category="category",
                output_format="gpd")
h3_bin.plot(
    column=stats,        # numeric column to base the colors on
    cmap='Spectral_r',        # color scheme (matplotlib colormap)
    legend=True,  
    linewidth=0.2          # boundary width (optional)
)

### Raster to H3

In [ ]:
from vgrid.conversion.raster2dggs.raster2h3 import raster2h3
from vgrid.utils.io import download_file

raster_url = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/raster/rgb.tif"
)
raster_file = download_file(raster_url)

raster_to_h3 =  raster2h3(raster_file,output_format="gpd")

# Visualize the output
import folium

m = folium.Map(tiles="CartoDB positron", max_zoom=28)

h3_layer = folium.GeoJson(
    raster_to_h3,
    style_function=lambda x: {
        "fillColor": f"rgb({x['properties']['band_1']}, {x['properties']['band_2']}, {x['properties']['band_3']})",
        "fillOpacity": 1,
        "color": "black",
        "weight": 1,
    },
    popup=folium.GeoJsonPopup(
        fields=["h3", "resolution", "band_1", "band_2", "band_3", "cell_area"],
        aliases=["H3 ID", "Resolution", "Band 1", "Band 2", "Band 3", "Area (m²)"],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
    ),
).add_to(m)

m.fit_bounds(h3_layer.get_bounds())

# Display the map
m

### H3 Generator

In [ ]:
from vgrid.generator.h3grid import h3grid

h3_grid = h3grid(resolution=0, output_format="gpd")
h3_grid.plot(edgecolor="white")

### H3 Statistics

In [ ]:
from vgrid.stats.h3stats import h3stats
h3stats()